# Gold-Palladium Dielectric Function Analysis

<style>
.nbviewer div.output_area {
  overflow-y: auto;
  max-height: 500px; /* or value of your choosing */
}
</style>

Hey Wiebke! I figured it'd be nice to actually use one of these Jupyter notebooks to do the entire data processing of the dielectric function of the AuPd alloys. This way you can see the whole thing, and comment where needed. And, I must admit, I'm a little giddy about starting these Jupyter notebooks a.t.m. :)

Let me start by describing the process that I followed to set up the calculations. As the DFT software package that I use only allows for periodic structures, I have tried to simulate the random positions of the Au-Pd alloy by generating a bunch of randomized structures. I actually wrote a little Python method for this:

```python

import os
from numpy.random import choice
from pymatgen.core import Structure

def generate_alloy(root_structure_file, alloy_element, mixing_ratio,
                   number_random):
    """

    Args:
        root_structure:
        alloy_element:
        mixing_ratio:
        number_random:

    Returns:

    """

    alloy = Structure.from_file(root_structure_file)

    random_list = []

    # Make a list of random combinations of site indices
    while len(random_list) < number_random:

        random_indices = list(choice(
            list(range(len(alloy))), round(len(alloy) * mixing_ratio), False
        ))

        if random_indices not in random_list:
            random_list.append(random_indices)

    # Get the current working directory
    directory = os.getcwd()

    # Set up the list of random structures
    for i, index_list in zip(range(len(random_list)), random_list):

        random_structure = alloy.copy()

        for site_index in index_list:
            random_structure.replace(site_index, alloy_element)

        filename = os.path.basename(root_structure_file).strip(
            ".json").strip(".cif") + "_rand" + str(i) + ".cif"

        random_structure.sort()
        random_structure.to(
            fmt="cif", filename=os.path.join(directory, filename)
        )
```

Please don't judge my lazy docstring. :P Basically, I give this script a supercell of Au, make Pd the `alloy_element`, and generate 10 (`number_random`) random structures for each %Pd (`mixing_ratio`). For each of these structures, I calculate the optical properties using DFT.

So far, I've just added an example output file (`test_diel.xml`) to the github repo to try and load it into this jupyter notebook. First step is to load the Vasprun object, that allows me to load the VASP output file that contains the dielectric function data.

In [1]:
from pymatgen.io.vasp.outputs import Vasprun

Pymatgen is an amazing python package with a lot of useful tools for material science. I've not tried ASE, but I'd be surprised if it has more to offer than pymatgen! Now, let's see if we can load the example .xml file:

In [2]:
output = Vasprun("test_diel.xml")

/Users/mbercx/Dropbox/python/envs/pymatgen/lib/python3.6/site-packages/pymatgen/io/vasp/outputs.py:877: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/mbercx/Dropbox/python/jupyter/wiebke_analysis
  " was found in {}".format(os.path.abspath(p)))


[Ew!](https://www.youtube.com/watch?v=FSNasZ5W_8A) A warning. All that red on my screen is making me sad. Let me see if I can fix this...

In [3]:
output = Vasprun("test_diel.xml", parse_potcar_file=False)

Much better! Now, let's see if we can get some dielectric function data from this baby.

In [4]:
output.dielectric

([0.0,
  0.0301,
  0.0603,
  0.0904,
  0.1205,
  0.1506,
  0.1808,
  0.2109,
  0.241,
  0.2712,
  0.3013,
  0.3314,
  0.3615,
  0.3917,
  0.4218,
  0.4519,
  0.482,
  0.5122,
  0.5423,
  0.5724,
  0.6026,
  0.6327,
  0.6628,
  0.6929,
  0.7231,
  0.7532,
  0.7833,
  0.8135,
  0.8436,
  0.8737,
  0.9038,
  0.934,
  0.9641,
  0.9942,
  1.0243,
  1.0545,
  1.0846,
  1.1147,
  1.1449,
  1.175,
  1.2051,
  1.2352,
  1.2654,
  1.2955,
  1.3256,
  1.3558,
  1.3859,
  1.416,
  1.4461,
  1.4763,
  1.5064,
  1.5365,
  1.5667,
  1.5968,
  1.6269,
  1.657,
  1.6872,
  1.7173,
  1.7474,
  1.7775,
  1.8077,
  1.8378,
  1.8679,
  1.8981,
  1.9282,
  1.9583,
  1.9884,
  2.0186,
  2.0487,
  2.0788,
  2.109,
  2.1391,
  2.1692,
  2.1993,
  2.2295,
  2.2596,
  2.2897,
  2.3198,
  2.35,
  2.3801,
  2.4102,
  2.4404,
  2.4705,
  2.5006,
  2.5307,
  2.5609,
  2.591,
  2.6211,
  2.6513,
  2.6814,
  2.7115,
  2.7416,
  2.7718,
  2.8019,
  2.832,
  2.8622,
  2.8923,
  2.9224,
  2.9525,
  2.9827,
  3.0128,
  3.

That sure is a lot of numbers! As you can tell by scrolling down every energy value corresponds to 6 dielectric function data elements, i.e. the `XX YY ZZ XY XZ YZ` components of the dielectric tensor. You can even see there is a little non-diagonal part of the dielectric tensor. In order to turn this into a function, we'll average the diagonal elements (and ignore the non-diagonal part). To do this, we need our trusty pal numpy!

In [5]:
import numpy as np
energy = np.array(output.dielectric[0])
diel_real = np.sum( np.array(output.dielectric[1])[:, 0:3], 1)
diel_imag = np.sum( np.array(output.dielectric[2])[:, 0:3], 1)

Numpy never lets you down! Now let's see if we can plot the dielectric function (Or, at least the interband part):

In [6]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(energy, diel_real)

ax.set(xlabel='Energy (eV)', ylabel='$\epsilon_1$',
       title='The real part of the dielectric function')
ax.grid()
ax.set_xlim(0, 2.5)
ax.set_ylim(-200, 800)
plt.show()

Not bad, not bad... *Looks at clock* AAHHHHH! I have to go to sleep. But I'll continue this later! These jupyter notebooks are pretty cool!